## Load and Customize a Pretrained Model

In [5]:
import torch
import torch.nn as nn
from torchvision import models, transforms

# Loads the pretrained version of the resnet CNN
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)

# Customize the final layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features # retrieves the features in the final fully connected (fc) layer 
                                # to define the new final layer correctly
model.fc = nn.Linear(num_ftrs, 4)  # output probabilities for 4 categories

# Move the model to the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print("Done loading pretrained model")

Done loading pretrained model


## Define Dataset and DataLoader 

In [10]:
from torch.utils.data import Dataset, DataLoader 
                            # represents data in a PyTorch compatible format
                            # efficiently loads batches of custom data
from PIL import Image # "Pillow" or Python Imaging Library
import pandas as pd
import os

class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0]) # retrieves value at specialized index and column 0
        image = Image.open(img_path).convert("RGB") # easy to convert to tensors

        # Crop the right half of the image
        width, height = image.size
        image = image.crop((width // 2, 0, width, height))
                        # right half, starts from top of image
        category = self.annotations.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        # creating a dictionary called sample
        sample = {
            'image': image, 
            'category': torch.tensor(category, dtype=torch.long)
        }
        return sample

transform = transforms.Compose([
    transforms.Resize((224, 224)), #resizes image to a square
    transforms.ToTensor() #converts image to a PyTorch tensor
])

train_dataset = CustomDataset('training.csv', 'training_images', transform=transform)
val_dataset = CustomDataset('validation.csv', 'training_images', transform=transform)
test_dataset = CustomDataset('testing.csv', 'training_images', transform=transform)

# DataLoader loads and manages data
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    

## Training Loop 

In [13]:
import torch.optim as optim # hosts optimization algorithms

criterion = nn.CrossEntropyLoss() # loss function - calculate negative log likelihood
optimizer = optim.Adam(model.parameters(), lr=0.001) # specifying Adam optimizer, learning rate, model parameters to update

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0 # initialized to 0 to track accumulated loss for that epoch
    for batch in train_loader:
        images = batch['image'].to(device)
        categories = batch['category'].to(device)

        optimizer.zero_grad() # reset gradients to 0 before each optimization step

        pred_categories = model(images) # feeds input images through neural network model

        loss = criterion(pred_categories, categories)
        loss.backward() # computes loss gradient and performs backpropogation to investigate each parameter's impact on loss
        optimizer.step() # update model parameters based on calculated gradients

        running_train_loss += loss.item() # calculate loss for the entire epoch

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad(): # don't need to calculate gradient for validation and testing
            for batch in val_loader:
                images = batch['image'].to(device)
                categories = batch['category'].to(device)
                
                pred_categories = model(images)
                
                loss = criterion(pred_categories, categories)
                running_val_loss += loss.item()
        
        print(
            f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_train_loss/len(train_loader)}, Validation Loss: {running_val_loss/len(val_loader)}"
        )

print("Training complete!")
        

Epoch 1/10, Training Loss: 0.21582384904225668, Validation Loss: 1.489869475364685
Epoch 1/10, Training Loss: 0.3890514241324531, Validation Loss: 1.6607062816619873
Epoch 1/10, Training Loss: 0.6084346108966403, Validation Loss: 1.601088285446167
Epoch 1/10, Training Loss: 0.8482757541868422, Validation Loss: 1.4109405279159546
Epoch 1/10, Training Loss: 1.0133864217334323, Validation Loss: 1.3991879224777222
Epoch 1/10, Training Loss: 1.1772932079103258, Validation Loss: 1.405877947807312
Epoch 1/10, Training Loss: 1.350628627671136, Validation Loss: 1.3928982615470886
Epoch 1/10, Training Loss: 1.5054910315407648, Validation Loss: 1.3881806135177612
Epoch 1/10, Training Loss: 1.6643512778811984, Validation Loss: 1.3890401124954224
Epoch 2/10, Training Loss: 0.3883405791388618, Validation Loss: 1.3919861912727356
Epoch 2/10, Training Loss: 0.54933676454756, Validation Loss: 1.4229751229286194
Epoch 2/10, Training Loss: 0.7049454185697768, Validation Loss: 1.436991572380066
Epoch 2/10

## Evaluation on Test Set

In [14]:
model.eval()
running_test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        categories = batch['category'].to(device)
        
        pred_categories = model(images)
        
        loss = criterion(pred_categories, categories)
        running_test_loss += loss.item()
        
        _, predicted_labels = torch.max(pred_categories, 1) # discard the max value, only need its indices
        correct_predictions += (predicted_labels == categories).sum().item() # looks at number of correct labels in the batch
        total_predictions += categories.size(0)

test_loss = running_test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f"Test Loss: {test_loss}, Test Accuracy: {accuracy * 100:.2f}%")

Test Loss: 1.2472459971904755, Test Accuracy: 20.00%


## Saving the Model

In [17]:
import torch

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

print("Model is saved.")

Model is saved.
